## Required installs/imports


In [1]:
!pip install pure-prng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.1 MB/s eta 0:00:00


In [2]:
# Necessary imports
from pure_prng_package import pure_prng
import numpy as np
import tensorflow as tf
from keras.layers import *
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import math
import time

## Classes to provide data generation/evaluation methods

In [3]:
# Class used to access my implmentations of PRNGs
class PRNGManagement():
  # Initialises object and sets the default seed
  def __init__(self, seed=0):
      self.seed = seed
      self.random_number:int = seed
      self.currentGenerator = self.zero_only_PRNG()

  # Method to seed the PRNG (seeds all PRNGs in class)
  def seed_PRNG(self, seed:int):
    self.seed:int = seed
    self.random_number:int = seed

  def set_generator(self, generatorMethod):
    self.currentGenerator = generatorMethod

  def bit_success(self, model, inputData, trueOutputs, sequence_length):
    """ 
    Method to evaluate the provided model and store the amount of
    successful predictions for each bit of the output
    :param model: keras model - Model used to generate predicitions
    :param trueOutputs: list[list[int]]- List containing the expected y outputs
    :param sequence_length: int - Length of generated binary string being predicited
    :return list[int] - Amount of successful predictions for each bit
    """
    # Set initial amount of successful predictions for each bit to zero
    successfulPredicts = [0]*sequence_length
    # Feeds the input data to the model and stores the predictions made
    predicted = (model.predict(inputData).round())
    # Iterate over all outputed data
    for testIndex in range(0, len(inputData)):
      # Iterate over each bit in output
      for i in range(sequence_length):
        # If the predicted bit matches the true bit value then increment the successful predicts for the current bit
        if predicted[testIndex][i] == trueOutputs[testIndex][i]: successfulPredicts[i] += 1
        # Prediction may be greater than 1 if the prediction is made with high certainity
        elif predicted[testIndex][i] > 1 and trueOutputs[testIndex][i] == 1: self.successfulPredicts[i] += 1

    return successfulPredicts
   

  def zero_only_PRNG(self, length=100):
    """ 
    Returns a binary string containing only 0 of specified length.
    Used to test for major flaws in models
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    return "0" * length        


  def alternating_bits_PRNG(self, length=100):
    """ 
    Returns output of a basic PRNG implementation that alernates each bit (010101)
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    # Use seed to determine the starting bit of the generated binary string
    self.seed = self.seed%2
    # Utilises efficent method to repeat a string pattern
    if (self.seed == 1):
        output = "10" * int(length/2)
    else:
        output = "01" * int(length/2)

    # Length of generated binary string is odd
    if (length%2 == 1):
      # Add final bit to string
      output += str(self.seed)
      # Set the new seed value
      if (self.seed == 0): self.seed = 1
      else: self.seed = 0
    
    return output


  def alternating_num_PRNG(self):
    """ 
    Returns output of a basic PRNG implementation that alernates between two binary strings
    :return string - generated binary string
    """
    # Use seed to determine the binary string to be returned
    self.seed = (self.seed+1)%2
    if (self.seed == 0):
      # Convert integer to a binary string 
      randomBinary = str(bin(1643712566))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary
    else:
      # Convert integer to a binary string 
      randomBinary = str(bin(2372817037))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary


  def basic_equation_based(self, mult:int, add:int, mod:int, leng:int) -> str:
    """ 
    Returns output of a very weak equation based PRNG implementation
    Expected to be predicted near perfectly
    :return string - generated binary string
    """
    # Generates random number using previous output as seed
    self.random_number = (mult * self.random_number + add) % mod
    # Converts generated number to a binary string
    bits_string = bin(self.random_number)[2:]
    # Returns binary string after using padding to ensure a length of 32
    return bits_string.zfill(leng)


  ## Different implmentations of equation based generators
  def basic_equation_based1(self) -> str:
    return self.basic_equation_based(20, 52, 2**32, 32)

  def basic_equation_based2(self) -> str:
    return self.basic_equation_based(36791, 83247, 2**32, 32)

  # Expects odd starting seed
  # Imeplementation of the PRNG 'RANDU' - Outdated PRNG
  def RANDU(self) -> str:
    return self.basic_equation_based(65539, 0, 2**31, 31)

  # Implementation of the Park Miller PRNG - Outdated PRNG
  def Park_Miller(self) -> str:
    return self.basic_equation_based(16807, 0, 2**31-1, 32)

  # Get stream of binary data form current generator
  def get_stream(self, number_of_blocks:int):
    stream = []
    for _ in range(number_of_blocks):
      # Converts generated binary string to list of ints
      block = [int(bit) for bit in self.currentGenerator()] 
      stream.extend(block)
    return stream

In [4]:
# Class used to manage the PRNGs offered by the PRNG library
class PRNGLibManagement():
  def __init__(self, PRNGType:str, outputLen:int):
    self.change_PRNG(PRNGType, outputLen)

  # Change the type of the current PRNG
  def change_PRNG(self, PRNGType:str, outputLen:int):
    self.outputLen = outputLen
    self.PRNGType = PRNGType
    self.currentGen = pure_prng(int(time.time()), prng_type=PRNGType).source_random_number()

  # Seed the current PRNG
  def seed_current(self, seed=int(time.time())):
    self.currentGen = pure_prng(int(seed), prng_type=self.PRNGType).source_random_number()

  # Generates an output from the curent PRNG
  def output_current(self):
    # Converts generated number to a binary string
    bits_string = bin(next(self.currentGen))[2:]
    # Returns binary string after using padding to ensure a consistent length
    return bits_string.zfill(self.outputLen)

  # Generates output from current PRNG as a list containing the integer bits
  def next_ints(self):
    # Converts generated binary string to list of ints
    return [int(bit) for bit in self.output_current()] 

  # Get stream of binary data form current generator
  def get_stream(self, number_of_blocks:int):
    stream = []
    for _ in range(number_of_blocks):
      block = self.next_ints()
      stream.extend(block)
    return stream

## Data generation function to be used during data production

In [5]:
# Creates object to use the 'QCG' PRNG
PRNGLibHandler = PRNGLibManagement("QCG", 256)

# Generation/seed functions for use
def generatorOutput(blockNum): return PRNGLibHandler.get_stream(blockNum)
def seedGenerator(seed): PRNGLibHandler.seed_current(seed)

## Setting parameters for data generation

In [6]:
### Setting parameters
# Sets paramemters for generating train/test data
num_blocks = 500
sequence_length = 256

# Calculates number of samples produced
num_samples = (num_blocks-1)*sequence_length

print("Number of samples: ", num_samples)

Number of samples:  127744


## Produces data for training/testing

In [7]:
# Seed generator
seedGenerator(23)
# Get stream data 
streamData = generatorOutput(num_blocks)

# Stores output in variable to allow the sample to be featued in both x and y data
X_data, Y_data = [], []

# Extracts stream data samples
for i in range(num_samples):
  X_data.append(streamData[i:i+sequence_length])
  Y_data.append([streamData[i+sequence_length]])

# Deletes stream data to save memory
del streamData

## Splitting data into training/testing sets

In [8]:
### Separate data into training/testing sets
# Percentage of data used for testing the created prediction model
testDataPerc = 0.2

x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = testDataPerc, random_state = 5)

del X_data
del Y_data

## Creation and training of different models

#### Dense + Normalisation + Dropout

In [9]:
# Sets parameters for model
epochs = 10

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Dense(sequence_length, input_shape=(sequence_length, ), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.2))
  model.add(BatchNormalization())
  model.add(Dense(sequence_length//2, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               65792     
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_2 (Batc  (None, 128)              5

In [10]:
batch_size = 100
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
1022/1022 [==============================] - 36s 28ms/step - loss: 0.7165 - accuracy: 0.5003 - val_loss: 0.6964 - val_accuracy: 0.5030
Epoch 2/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6919 - accuracy: 0.5241 - val_loss: 0.6960 - val_accuracy: 0.5026
Epoch 3/10
1022/1022 [==============================] - 6s 6ms/step - loss: 0.6900 - accuracy: 0.5350 - val_loss: 0.6985 - val_accuracy: 0.4975
Epoch 4/10
1022/1022 [==============================] - 6s 6ms/step - loss: 0.6873 - accuracy: 0.5431 - val_loss: 0.7018 - val_accuracy: 0.4988
Epoch 5/10
1022/1022 [==============================] - 8s 8ms/step - loss: 0.6849 - accuracy: 0.5500 - val_loss: 0.7004 - val_accuracy: 0.5032
Epoch 6/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6814 - accuracy: 0.5625 - val_loss: 0.7068 - val_accuracy: 0.4994
Epoch 7/10
1022/1022 [==============================] - 7s 6ms/step - loss: 0.6765 - accuracy: 0.5740 - val_loss: 0.7110 - val_accurac

In [11]:
model.save("QCGDND1.h5")

#### Long Short-Term Memory

In [12]:
# Sets parameters for model
epochs = 10

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(LSTM(int(sequence_length), input_shape=(sequence_length, 1), return_sequences=True))
  model.add(LSTM(int(sequence_length/1.5), return_sequences=True))
  model.add(LSTM(int(sequence_length/2)))
  model.add(Dense(1, activation='sigmoid'))


  model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

  model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 256, 256)          264192    
                                                                 
 lstm_1 (LSTM)               (None, 256, 170)          290360    
                                                                 
 lstm_2 (LSTM)               (None, 128)               153088    
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 707,769
Trainable params: 707,769
Non-trainable params: 0
_________________________________________________________________


In [13]:
batch_size = 100
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
1022/1022 [==============================] - 104s 95ms/step - loss: 0.6933 - accuracy: 0.4981 - val_loss: 0.6931 - val_accuracy: 0.5003
Epoch 2/10
1022/1022 [==============================] - 76s 74ms/step - loss: 0.6932 - accuracy: 0.5003 - val_loss: 0.6931 - val_accuracy: 0.5027
Epoch 3/10
1022/1022 [==============================] - 75s 74ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6933 - val_accuracy: 0.5006
Epoch 4/10
1022/1022 [==============================] - 75s 73ms/step - loss: 0.6932 - accuracy: 0.4972 - val_loss: 0.6931 - val_accuracy: 0.5006
Epoch 5/10
1022/1022 [==============================] - 76s 75ms/step - loss: 0.6932 - accuracy: 0.5016 - val_loss: 0.6932 - val_accuracy: 0.5006
Epoch 6/10
1022/1022 [==============================] - 76s 74ms/step - loss: 0.6932 - accuracy: 0.4988 - val_loss: 0.6932 - val_accuracy: 0.4994
Epoch 7/10
1022/1022 [==============================] - 76s 74ms/step - loss: 0.6932 - accuracy: 0.5021 - val_loss: 0.6931 

In [14]:
model.save("QCGLTSM1.h5")

#### Conv1d

In [15]:
# Sets parameters for produced model
epochs = 10

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(sequence_length,1)))
  model.add(Flatten())
  model.add(Dense(sequence_length, input_shape=(sequence_length, ), activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 254, 128)          512       
                                                                 
 flatten (Flatten)           (None, 32512)             0         
                                                                 
 dense_4 (Dense)             (None, 256)               8323328   
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 8,324,097
Trainable params: 8,324,097
Non-trainable params: 0
_________________________________________________________________


In [16]:
batch_size = 100
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
1022/1022 [==============================] - 35s 29ms/step - loss: 0.6988 - accuracy: 0.4961 - val_loss: 0.6931 - val_accuracy: 0.4991
Epoch 2/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6929 - accuracy: 0.5101 - val_loss: 0.6933 - val_accuracy: 0.4976
Epoch 3/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6923 - accuracy: 0.5155 - val_loss: 0.6941 - val_accuracy: 0.4983
Epoch 4/10
1022/1022 [==============================] - 8s 7ms/step - loss: 0.6908 - accuracy: 0.5240 - val_loss: 0.6942 - val_accuracy: 0.5034
Epoch 5/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6873 - accuracy: 0.5399 - val_loss: 0.6960 - val_accuracy: 0.5022
Epoch 6/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6756 - accuracy: 0.5729 - val_loss: 0.7053 - val_accuracy: 0.5040
Epoch 7/10
1022/1022 [==============================] - 7s 7ms/step - loss: 0.6376 - accuracy: 0.6327 - val_loss: 0.7391 - val_accurac

In [17]:
model.save("QCGConv1D1.h5")

#### Conv1d + Normalisation + Dropout

In [18]:
# Sets parameters for produced model
epochs = 10

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(sequence_length,1)))
  model.add(Flatten())
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.1))
  model.add(Dense(sequence_length, input_shape=(sequence_length, ), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 254, 128)          512       
                                                                 
 flatten_1 (Flatten)         (None, 32512)             0         
                                                                 
 batch_normalization_3 (Batc  (None, 32512)            130048    
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 32512)             0         
                                                                 
 dense_6 (Dense)             (None, 256)               8323328   
                                                                 
 batch_normalization_4 (Batc  (None, 256)              1024      
 hNormalization)                                      

In [19]:
batch_size = 100
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
1022/1022 [==============================] - 34s 31ms/step - loss: 0.7638 - accuracy: 0.4990 - val_loss: 0.6961 - val_accuracy: 0.5031
Epoch 2/10
1022/1022 [==============================] - 10s 10ms/step - loss: 0.6900 - accuracy: 0.5352 - val_loss: 0.6999 - val_accuracy: 0.5027
Epoch 3/10
1022/1022 [==============================] - 11s 10ms/step - loss: 0.6828 - accuracy: 0.5578 - val_loss: 0.7047 - val_accuracy: 0.4993
Epoch 4/10
1022/1022 [==============================] - 11s 10ms/step - loss: 0.6684 - accuracy: 0.5900 - val_loss: 0.7207 - val_accuracy: 0.4988
Epoch 5/10
1022/1022 [==============================] - 10s 10ms/step - loss: 0.6458 - accuracy: 0.6251 - val_loss: 0.7275 - val_accuracy: 0.5008
Epoch 6/10
1022/1022 [==============================] - 10s 10ms/step - loss: 0.6135 - accuracy: 0.6626 - val_loss: 0.7653 - val_accuracy: 0.4997
Epoch 7/10
1022/1022 [==============================] - 10s 10ms/step - loss: 0.5660 - accuracy: 0.7070 - val_loss: 0.8735 -

##### Training with additioanl data blocks

In [37]:
batch_size = 100
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
6142/6142 [==============================] - 195s 31ms/step - loss: 0.7049 - accuracy: 0.5001 - val_loss: 0.6948 - val_accuracy: 0.5003
Epoch 2/10
6142/6142 [==============================] - 60s 10ms/step - loss: 0.6944 - accuracy: 0.5015 - val_loss: 0.6941 - val_accuracy: 0.5031
Epoch 3/10
6142/6142 [==============================] - 60s 10ms/step - loss: 0.6934 - accuracy: 0.5015 - val_loss: 0.6933 - val_accuracy: 0.4989
Epoch 4/10
6142/6142 [==============================] - 61s 10ms/step - loss: 0.6932 - accuracy: 0.5031 - val_loss: 0.6933 - val_accuracy: 0.5013
Epoch 5/10
6142/6142 [==============================] - 61s 10ms/step - loss: 0.6931 - accuracy: 0.5040 - val_loss: 0.6932 - val_accuracy: 0.4995
Epoch 6/10
6142/6142 [==============================] - 61s 10ms/step - loss: 0.6931 - accuracy: 0.5058 - val_loss: 0.6933 - val_accuracy: 0.4992
Epoch 7/10
6142/6142 [==============================] - 61s 10ms/step - loss: 0.6930 - accuracy: 0.5079 - val_loss: 0.6933 

In [ ]:
model.save("QCGCND1.h5")

## Concatenated samples with best model

In [ ]:
## Example for single model

# Sets paramemters for generating train/test data
num_blocks = 1000
sequence_length = 256
samplesConcatenated = 2
num_samples = (num_blocks-1)*sequence_length-sequence_length*(samplesConcatenated-1)

print("Number of samples: ", num_samples)

# Seed generator
seedGenerator(23)
# Get stream data 
streamData = generatorOutput(num_blocks)

# Stores output in variable to allow the sample to be featued in both x and y data
X_data = []
Y_data = []

for i in range(0, num_samples):
  X_data.append(streamData[i:i+sequence_length*samplesConcatenated])
  Y_data.append([streamData[i+sequence_length*samplesConcatenated]])

del streamData

x_train, x_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.3, random_state=32)
del X_data
del Y_data

Number of samples:  255488


In [ ]:
# Sets parameters for produced model
epochs = 10

# Strategy to utilise GPU 
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

# Model compilation using GPU
with strategy.scope():
  model = Sequential()
  model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(sequence_length*samplesConcatenated,1)))
  model.add(Flatten())
  model.add(BatchNormalization())
  model.add(Dropout(rate=0.1))
  model.add(Dense(sequence_length, input_shape=(sequence_length, ), activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 510, 128)          512       
                                                                 
 flatten_1 (Flatten)         (None, 65280)             0         
                                                                 
 batch_normalization_2 (Batc  (None, 65280)            261120    
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 65280)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               16711936  
                                                                 
 batch_normalization_3 (Batc  (None, 256)              1024      
 hNormalization)                                      

In [ ]:
batch_size = 100
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
1789/1789 [==============================] - 177s 93ms/step - loss: 0.7332 - accuracy: 0.5013 - val_loss: 0.6945 - val_accuracy: 0.5004
Epoch 2/10
1789/1789 [==============================] - 31s 17ms/step - loss: 0.6931 - accuracy: 0.5116 - val_loss: 0.6942 - val_accuracy: 0.4987
Epoch 3/10
1789/1789 [==============================] - 30s 17ms/step - loss: 0.6922 - accuracy: 0.5155 - val_loss: 0.6979 - val_accuracy: 0.5025
Epoch 4/10
1789/1789 [==============================] - 32s 18ms/step - loss: 0.6912 - accuracy: 0.5234 - val_loss: 0.6944 - val_accuracy: 0.5007
Epoch 5/10
1789/1789 [==============================] - 30s 17ms/step - loss: 0.6900 - accuracy: 0.5327 - val_loss: 0.6994 - val_accuracy: 0.5016
Epoch 6/10
1789/1789 [==============================] - 31s 17ms/step - loss: 0.6841 - accuracy: 0.5506 - val_loss: 0.7006 - val_accuracy: 0.5028
Epoch 7/10
1789/1789 [==============================] - 30s 17ms/step - loss: 0.6734 - accuracy: 0.5794 - val_loss: 0.7203 

In [ ]:
# Not recommended due to large size of model
model.save("QCGCNDConcat1.h5")

#### Graphing current model history

In [ ]:
print("Plotting graph")
# Plot training & validation loss values
# Plots loss data
plt.plot(range(1, epochs+1), history.history['loss'])
plt.plot(range(1, epochs+1), history.history['val_loss'])
# Plots a label for each axis
plt.ylabel('Loss Value')
plt.xlabel('Epoch Number')
# Plots the title
plt.title('Model loss over epochs')
# Plots the legend for each loss
plt.legend(['Train', 'Validation'], loc='upper right')

# Sets grid for plot
plt.grid(linestyle='--', alpha=0.4)
# Ensures the x-axis only contains integers
plt.xticks(range(1, epochs+1))


# Displays plot
plt.show()